# Setup

## Installs

In [1]:
# Segmentation
# ! pip install --upgrade mxnet
# ! pip install torch==1.6.0+cpu torchvision==0.7.0+cpu -f https://download.pytorch.org/whl/torch_stable.html
# ! pip install --upgrade gluoncv

# Edge detection
# ! conda install scikit-image

# Depth estimation
# ! pip install -q transformers

## Imports

In [2]:
import numpy as np
import pandas as pd
import time
import cv2
from os import listdir
from matplotlib import pyplot as plt

In [3]:
# Import custom methods
import general_methods
import semantic_segmentation
import edge_detection
import depth_estimation
import geometry
import transforms

c:\Users\Kim\anaconda3\envs\pipeline1\lib\site-packages\gluoncv\__init__.py:40: UserWarning: Both `mxnet==1.7.0` and `torch==1.6.0+cpu` are installed. You might encounter increased GPU memory footprint if both framework are used at the same time.
  warnings.warn(f'Both `mxnet=={mx.__version__}` and `torch=={torch.__version__}` are installed. '


In [4]:
# Get pre-trained models
seg_model = semantic_segmentation.get_pretrained_model()

# Data Collection

In [5]:
def pipeline_data_collection(filename, wallpaper_filename, use_ceiling=False, use_floor=False):
    start_time = time.time()
    used_ceiling, used_floor = False, False

    input_pil = general_methods.import_and_resize(filename)
    input_img = general_methods.import_mx_image("images/outputs/intermediate-outputs/resized-input.png")
    input_cv2 = general_methods.import_cv2_image("images/outputs/intermediate-outputs/resized-input.png")
    height = input_cv2.shape[0]
    width = input_cv2.shape[1]
    size = (width, height)

    output_list = [filename.split("/")[-1], height, width]

    # Segmentation
    mmask = semantic_segmentation.get_segementation(input_img, seg_model)
    labels = general_methods.get_labels_string(mmask, mmask.shape[0])
    walls = general_methods.find_colour_indices(labels, "0.47058824,0.47058824,0.47058824,1.0")
    other = general_methods.find_not_colour_indices(labels, "0.47058824,0.47058824,0.47058824,1.0")
    segmented_input = semantic_segmentation.remove_inds(input_img, [other], [0, 0, 0])
    ceiling_x, _ = np.where(labels == "0.47058824,0.47058824,0.3137255,1.0")

    # Depth estimation
    depth_image = depth_estimation.estimate_depth(input_pil)

    if (ceiling_x.size > (0.01 * (height * width))) and use_ceiling:
        used_ceiling = True
        non_ceil_x, non_ceil_y = np.where((labels != "0.47058824,0.47058824,0.47058824,1.0") & (labels != "0.47058824,0.47058824,0.3137255,1.0"))
        non_ceil = np.array([non_ceil_x, non_ceil_y])

        input_copy = np.ones((input_cv2.shape[0], input_cv2.shape[1], 3))
        input_copy = input_copy.astype(float)
        input_copy[non_ceil[0], non_ceil[1]] = np.array([255, 255, 255], dtype=float)
        input_copy[walls[0], walls[1]] = np.array([255, 255, 255], dtype=float)
        input_copy = np.clip(input_copy, 0, 255)

        segmented_input_2 = input_copy.astype(np.uint8)

        for i in range(input_cv2.shape[1]):
            for j in range(input_cv2.shape[0]-1, -1, -1):
                if np.array_equal(segmented_input_2[j][i], [1, 1, 1]):
                    segmented_input_2[:j, i] = np.array([1, 1, 1], dtype=float)
                    break

        edge_map = edge_detection.detect_edges(segmented_input_2)
        edge_map = np.asarray(edge_map.convert("RGB"))

        hough_img = edge_detection.hough_transform(edge_map, input_cv2.shape[0], 10, 0.001, 0.5)
        
        corners = depth_estimation.get_harris_corners(hough_img)

        labels_red = general_methods.get_labels_string(corners, 40)
        red = general_methods.find_colour_indices(labels_red, "255.0,0.0,0.0")
        
        temp2 = corners.copy()
        temp2[red[0], red[1]] = np.array([0, 0, 0], dtype=float)

        ceiling_colours = general_methods.get_labels_string(temp2, temp2.shape[0])
        _, ceiling_corners = np.where(ceiling_colours == "255.0,0.0,0.0")
        corner_inds = geometry.find_corners(ceiling_corners, ceiling_corners, width)

        if corner_inds.size == 0:
            used_ceiling = False
            
            # Edge detection
            edge_map = edge_detection.detect_edges(input_cv2)
            segmented_edges = edge_detection.get_segmented_edges(edge_map, walls)

            # Find corners
            hough_img = edge_detection.hough_transform(segmented_edges, height, 50, 0.03, 0.02)
            vertical_lines = edge_detection.get_vertical_lines(hough_img)
            hough_colours = general_methods.get_labels_string(vertical_lines, vertical_lines.shape[0])
            hough_corners = edge_detection.get_hough_corners(hough_colours)
            mean_depth = depth_estimation.get_mean_depths(depth_image, other)
            matrix = general_methods.get_matrix(mean_depth)
            matrix_corners = depth_estimation.get_harris_corners(matrix)
            harris_colours = general_methods.get_labels_string(matrix_corners, matrix_corners.shape[0])
            _, harris_corners = np.where(harris_colours == "255,0,0")
            corner_inds = geometry.find_corners(hough_corners, harris_corners, width)
    else:
        # Edge detection
        edge_map = edge_detection.detect_edges(input_cv2)
        segmented_edges = edge_detection.get_segmented_edges(edge_map, walls)

        # Find corners
        hough_img = edge_detection.hough_transform(segmented_edges, height, 50, 0.03, 0.02)
        vertical_lines = edge_detection.get_vertical_lines(hough_img)
        hough_colours = general_methods.get_labels_string(vertical_lines, vertical_lines.shape[0])
        hough_corners = edge_detection.get_hough_corners(hough_colours)
        mean_depth = depth_estimation.get_mean_depths(depth_image, other)
        matrix = general_methods.get_matrix(mean_depth)
        matrix_corners = depth_estimation.get_harris_corners(matrix)
        harris_colours = general_methods.get_labels_string(matrix_corners, matrix_corners.shape[0])
        _, harris_corners = np.where(harris_colours == "255,0,0")
        corner_inds = geometry.find_corners(hough_corners, harris_corners, width)
    
    ## Add corners to output 
    corner_inds = np.sort(corner_inds)
    for i in range(5):
        if i < len(corner_inds):
            output_list.append(corner_inds[i])
        else:
            output_list.append(np.nan)

    only_walls = geometry.create_wall_corner_map(segmented_input, other, walls, corner_inds)

    # Find room geometry
    contours = geometry.find_contours(only_walls)
    corner_adj_geom = geometry.find_walls(contours, corner_inds)
    new_geom = geometry.find_quadrilaterals(corner_adj_geom, width)
    new_corner_geom = geometry.remove_nested_geometry(new_geom)
    new_corner_geom = geometry.move_edges_to_corners(new_corner_geom, corner_inds, width)
    
    # new_corner_geom = np.array([transforms.order_corner_points(geom) for geom in new_corner_geom])

    floor = general_methods.find_colour_indices(labels, "0.3137255,0.19607843,0.19607843,1.0")
    intersection_ys = [] # needed otherwise it doesn't work when adding the intersection ys to output_list
    if (floor[0].size > (0.05 * (height * width))) and use_floor:
        used_floor = True
        corner_inds = np.append(corner_inds, [0, width-1])
        intersection_ys = geometry.find_floor_intersection(walls, floor, np.array(depth_image), corner_inds)
        new_corner_geom = geometry.set_geom_corner_intersection(new_corner_geom, corner_inds, intersection_ys)

    ## Add corner floor intersection points
    for i in range(5):
        if i < len(intersection_ys):
            output_list.append(intersection_ys[i])
        else:
            output_list.append(np.nan)


    # Perspective transform
    wallpaper = general_methods.import_cv2_image(wallpaper_filename)
    result_1, result_2 = transforms.get_transformed_wallpaper(new_corner_geom, height, width, size, wallpaper)

    # Create final image
    final_mask, extra_mask = transforms.get_wall_mask(new_corner_geom, height, width, walls)
    final_output_1, final_output_2 = transforms.combine_wallpaper_and_input(input_cv2, final_mask, extra_mask, result_1, result_2, walls)

    output_list.append(time.time() - start_time)

    walls_map = np.zeros((height, width))
    walls_map[walls[0], walls[1]] = 1

    overlap = walls_map + final_mask
    overlap_perc = np.where(overlap == 2)[0].shape[0] / np.where(overlap >= 1)[0].shape[0]
    output_list.append(overlap_perc)
    output_list.append(used_ceiling)
    output_list.append(used_floor)

    output_pd = pd.DataFrame([output_list], columns=["filename", "height", "width",
                                            "corner-1", "corner-2", "corner-3",
                                            "corner-4", "corner-5", "co-fl-in-1",
                                            "co-fl-in-2", "co-fl-in-3", "co-fl-in-4",
                                            "co-fl-in-5", "time-elapsed", "wall-coverage-error",
                                            "used-ceiling", "used-floor"])
    return output_pd, final_output_1, final_output_2

In [10]:
# output_pd, final_output_1, _ = pipeline_data_collection("images/inputs/rooms/test-set/ADE_val_00000119.jpg", "images/inputs/wallpaper/check-even.jpg", True, True)
# cv2.imwrite(f"data-collection/outputs/final-images/ADE_val_00000119.jpg", final_output_1)
# # dining-room5.jpg
# # bedroom-6.jpg
# # ADE_val_00001083.jpg
# # ADE_val_00000474.jpg
# # ADE_val_00000119.jpg

True

<Figure size 640x480 with 0 Axes>

In [ ]:
# Loop over all images in input folder
folder_dir = "images/inputs/rooms/test-set"
wallpaper_filename = "images/inputs/wallpaper/check-even.jpg"
full_output_pd = pd.DataFrame(data=[], columns=["filename", "height", "width",
                                            "corner-1", "corner-2", "corner-3",
                                            "corner-4", "corner-5", "co-fl-in-1",
                                            "co-fl-in-2", "co-fl-in-3", "co-fl-in-4",
                                            "co-fl-in-5", "time-elapsed", "wall-coverage-error",
                                            "used-ceiling", "used-floor"]) 
output_file = "data-collection/outputs/performance-data-true-false.csv"
output_progress = []
try:
    full_output_pd = pd.read_csv(output_file)
    output_progress = full_output_pd["filename"].tolist()
except: 
    pass
print(output_progress)

for image in listdir(folder_dir):
    output_pd = None
    if (image not in output_progress):
        print(image)
        filename = "images/inputs/rooms/test-set/" + image
        try:
            output_pd, final_output_1, _ = pipeline_data_collection(filename, wallpaper_filename, True, False)
        except Exception as e:
            print(e)
            output_pd = pd.DataFrame(data=[[image, "FAILED", "FAILED", "FAILED", "FAILED", "FAILED", "FAILED",
                                        "FAILED", "FAILED", "FAILED", "FAILED", "FAILED", "FAILED",
                                        "FAILED", "FAILED", "FAILED", "FAILED"]],
                         columns=["filename", "height", "width",
                                    "corner-1", "corner-2", "corner-3",
                                    "corner-4", "corner-5", "co-fl-in-1",
                                    "co-fl-in-2", "co-fl-in-3", "co-fl-in-4",
                                    "co-fl-in-5", "time-elapsed", "wall-coverage-error",
                                    "used-ceiling", "used-floor"])
        
        full_output_pd = pd.concat((full_output_pd, output_pd))
        cv2.imwrite(f"data-collection/outputs/final-images-true-false/{image}", final_output_1)

    full_output_pd.to_csv(output_file, index=False)

In [ ]:
# pd.read_csv("data-collection/groundtruths/target-data.csv", index_col=None)